In [1]:
import numpy as np
import tensorflow as tf
import os
from keras import regularizers
from keras.models import Sequential
from keras.layers.convolutional import Convolution2D, MaxPooling2D,AveragePooling2D
from keras.layers import Activation, Flatten, Dense, Dropout,SpatialDropout2D
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
import cv2
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D,ZeroPadding2D,AveragePooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.layers import Input,add
from keras.models import Model
from keras.layers import concatenate 
from keras.layers import GlobalAveragePooling2D

Using TensorFlow backend.


In [2]:
#loading dataset
from keras.datasets import cifar10
(train_features, train_labels), (test_features, test_labels) = cifar10.load_data()
num_train, img_channels, img_rows, img_cols =  train_features.shape
num_test, _, _, _ =  test_features.shape
num_classes = len(np.unique(train_labels))

170500096/170498071 [==============================] - 4s 0us/step


In [0]:
train_features = train_features.astype('float32')/255
test_features = test_features.astype('float32')/255
# convert class labels to binary class labels
train_labels = np_utils.to_categorical(train_labels, num_classes)
test_labels = np_utils.to_categorical(test_labels, num_classes)

In [0]:
def get_cutout_eraser_and_random_crop(p=0.5,s_l=0.05,s_h=0.3,r_1=0.3,r_2=1/0.3,max_erasers_per_image=1,pixel_level=True,random_crop_size=(32,32),padding_pixels=4):
  
  assert max_erasers_per_image>=1 
  def eraser(input_img):
        v_l = np.min(input_img)
        v_h = np.max(input_img)
        img_h, img_w, img_c = input_img.shape
        p_1 = np.random.rand()

        if p_1 > p:
            return input_img
        mx = np.random.randint(1,max_erasers_per_image+1)
        for i in range(mx):
          while True:
              s = np.random.uniform(s_l, s_h) * img_h * img_w
              r = np.random.uniform(r_1, r_2)
              w = int(np.sqrt(s / r))
              h = int(np.sqrt(s * r))
              left = np.random.randint(0, img_w)
              top = np.random.randint(0, img_h)

              if left + w <= img_w and top + h <= img_h:
                  break

          if pixel_level:
              c = np.random.uniform(v_l, v_h, (h, w, img_c))
          else:
              c = np.random.uniform(v_l, v_h)

          input_img[top:top + h, left:left + w, :] = c
        return input_img

    
  def random_crop(input_image):
    assert input_image.shape[2]==3

    #pad for 4 pixels
    img = cv2.copyMakeBorder(input_image,padding_pixels,padding_pixels,padding_pixels,padding_pixels,cv2.BORDER_REPLICATE)
    height , width =img.shape[0],img.shape[1]
    dy,dx = random_crop_size
    x = np.random.randint(0,width - dx + 1)
    y = np.random.randint(0,height - dy + 1)
    return img[y:(y+dy),x:(x+dx),:]

  def preprocess_image(input_image):
    return eraser(random_crop(input_image))
  
  return preprocess_image
      

In [0]:
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

datagen = ImageDataGenerator(horizontal_flip=0.5,preprocessing_function=get_cutout_eraser_and_random_crop())
test_datagen = ImageDataGenerator(horizontal_flip=True,vertical_flip=True)
#test_datagen.fit(train_features)

In [0]:
datagen.mean = np.array([0.4914, 0.4822, 0.4465], dtype=np.float32).reshape((1,1,3)) # ordering: [R, G, B]
datagen.std = np.array([0.2023, 0.1994, 0.2010], dtype=np.float32).reshape((1,1,3)) # ordering: [R, G, B

#Resnet Model Creation

In [0]:
def ResNetBlock(input_layer, channels,stride=1):
  
  bn_1 = BatchNormalization(momentum=0.9,epsilon=1e-5)(input_layer)
  activation_layer_b1 = Activation('relu')(bn_1)
  block_layer_1 = Convolution2D(channels, (3,3), padding='same',strides=stride)(activation_layer_b1)
  
  bn_2 = BatchNormalization(momentum=0.9,epsilon=1e-5)(block_layer_1)
  activation_layer_b2 = Activation('relu')(bn_2) 
  block_layer_2 = Convolution2D(channels, (3,3), padding='same')(activation_layer_b2)
   
  
  return block_layer_2
  

In [8]:
from keras.layers import Input, add, GlobalAveragePooling2D, Dense
from keras.models import Model

inputs =Input(shape=(32, 32, 3))

x1 = Convolution2D(32, (3, 3),padding='same')(inputs)   #32x32 
activation_x1 = Activation('relu')(x1)
bn1 = BatchNormalization(momentum=0.9,epsilon=1e-5)(activation_x1)

x2 = Convolution2D(32, (3, 3),padding='same')(bn1)   #32x32 
activation_x2 = Activation('relu')(x2)
bn2 = BatchNormalization()(activation_x2)

# x3 = Convolution2D(64, (3, 3),padding='same')(bn2)   #32x32 
# activation_x3 = Activation('relu')(x3)
# bn3 = BatchNormalization()(activation_x3)


##block 1

blk1 = ResNetBlock(bn2,32)  ##32x32
z1 = add([blk1,bn2])

blk1_c = ResNetBlock(z1,32)
z1_c = add([blk1_c,z1])

drp1 = SpatialDropout2D(0.05)(z1_c)

##block 2

blk2 = ResNetBlock(drp1,64,stride=2)
one_blk = Convolution2D(64, (1, 1), padding='same',strides=2)(drp1)
z2 = add([blk2,one_blk])

blk2_c = ResNetBlock(z2,64)
z2_c = add([blk2_c,z2])

drp2 = SpatialDropout2D(0.05)(z2_c)

##block3

blk3 = ResNetBlock(drp2,128)
one_blk_1 = Convolution2D(128, (1, 1), padding='same')(drp2)
z3 = add([blk3,one_blk_1])

blk3_c = ResNetBlock(z3,128)
z3_c = add([blk3_c,z3])

# drp3 = SpatialDropout2D(0.05)(z3_c)
##block4

# blk4 = ResNetBlock(drp3,256,stride=2)
# one_blk_2 = Convolution2D(256, (1, 1), padding='same',strides=2)(drp3)
# z4 = add([blk4,one_blk_2])

# blk4_c = ResNetBlock(z4,256)
# z4_c = add([blk4_c,z4])


avg_pool_layer = GlobalAveragePooling2D()(z3_c)

#flatten_layer = Flatten()(avg_pool_layer)

fc_layer = Dense(10, activation='softmax')(avg_pool_layer)


model = Model(inputs=inputs, outputs= fc_layer)



W0805 12:00:26.328115 140136275920768 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0805 12:00:26.373179 140136275920768 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0805 12:00:26.380679 140136275920768 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0805 12:00:26.422042 140136275920768 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0805 12:00:26.423027 1401362759

In [9]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 32)   896         input_1[0][0]                    
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 32)   0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 32)   128         activation_1[0][0]               
__________________________________________________________________________________________________
conv2d_2 (

In [0]:
#from one_cycle_lr import LRFinder
from one_cycle_lr import LRFinder
num_samples= train_features.shape[0]
batch_size =512
num_epoch=30
max_lr= 0.1


#Best LR would be 0.01

In [12]:
from keras import optimizers

opt = optimizers.SGD(decay=5e-4)
model.compile(optimizer=opt , loss='categorical_crossentropy', metrics=['accuracy'])

W0805 12:01:14.854370 140136275920768 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [0]:
from one_cycle_lr import OneCycleLR

lr_manager = OneCycleLR(num_samples, num_epoch, batch_size, max_lr,
                        end_percentage=0.1, scale_percentage=None,
                        maximum_momentum=0.95, minimum_momentum=0.85)
                        


In [0]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [0]:
filepath = "Resnet-13.hdf5"


checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')


# Train the model
model_info = model.fit_generator(datagen.flow(train_features, train_labels, batch_size = 512),
                                 samples_per_epoch = train_features.shape[0], nb_epoch = 30, 
                                 validation_data = (test_features, test_labels), verbose=1,callbacks=[checkpoint,lr_manager])


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., validation_data=(array([[[..., verbose=1, callbacks=[<keras.ca..., steps_per_epoch=97, epochs=30)`
  # Remove the CWD from sys.path while we load stuff.
W0805 12:01:33.139751 140136275920768 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_su

Epoch 1/30
97/97 [==============================] - 31s 318ms/step - loss: 1.8885 - acc: 0.3002 - val_loss: 2.0573 - val_acc: 0.3485

Epoch 00001: val_acc improved from -inf to 0.34850, saving model to Resnet-13.hdf5
 - lr: 0.01660 - momentum: 0.94 
Epoch 2/30
97/97 [==============================] - 25s 259ms/step - loss: 1.5333 - acc: 0.4400 - val_loss: 2.2814 - val_acc: 0.3388

Epoch 00002: val_acc did not improve from 0.34850
 - lr: 0.02327 - momentum: 0.94 
Epoch 3/30
97/97 [==============================] - 25s 253ms/step - loss: 1.3311 - acc: 0.5151 - val_loss: 1.4571 - val_acc: 0.5129

Epoch 00003: val_acc improved from 0.34850 to 0.51290, saving model to Resnet-13.hdf5
 - lr: 0.02994 - momentum: 0.93 
Epoch 4/30
97/97 [==============================] - 25s 256ms/step - loss: 1.2007 - acc: 0.5664 - val_loss: 1.2891 - val_acc: 0.5613

Epoch 00004: val_acc improved from 0.51290 to 0.56130, saving model to Resnet-13.hdf5
 - lr: 0.03661 - momentum: 0.92 
Epoch 5/30
97/97 [=========